# Analyzing a proposal for NJ districting

This notebook provides an interactive plot demonstrating the likely partisan performance of a map drawn according to the rules laid out in [SCR43](https://www.njleg.state.nj.us/2018/Bills/SCR/43_I1.PDF).

The relevant text is as follow, with particularly relevant bits bolded:

>c. The Commission shall only certify a plan to establish legislative districts that ensures fair representation such that **each of the two major political parties has an equal number of districts more favorable to that party**. A District shall be more favorable to a political party if the percentage of total votes received in that district in all Statewide general elections by that party over the preceding decade of the offices of United States President, United States Senator, and Governor exceeds the percentage of total votes that party received in the average district in the plan, weighting each district equally.
>
>d. The Commission shall only certify a plan to establish legislative districts that ensures that **at least 25 percent of all districts are competitive districts, which shall mean a district that is more favorable to either major political party by no more than five percentage points of the average district in the plan. For each competitive district in which the percentage of total votes for a major political party exceeds that party’s percentage of total votes in the average district, there shall be a corresponding district in which that party’s percentage of total votes is less than the major party’s percentage of total votes in the average district by approximately the same percentage.**

## Hit the "Run" button at the top and scroll down to play with the interactive!

In [132]:
import numpy as np
import gerrymetrics as g
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
%config InlineBackend.figure_format = 'svg'
import ipywidgets as wdgt

avgcolor = 'forestgreen'
curvecolor = 'red'

def make_plot(avg=.54, comp_prop=.25, comp_range=.05, N=4000, show_hist=True, use_percents=False):
    N_comp = round(N*comp_prop)

    comp_votes_hi = np.random.rand(round(N_comp/2))*comp_range + avg
    comp_votes_lo = -comp_votes_hi + 2*avg

    other_votes_hi = np.random.rand(round((N-N_comp)/2))*(1-avg-comp_range) + avg + comp_range
    # other_votes_lo = np.random.rand(round((N-N_comp)/2))*(avg-comp_range)
    other_votes_lo = -other_votes_hi + 2*avg

    Dvotes = np.concatenate((comp_votes_hi, comp_votes_lo, other_votes_hi, other_votes_lo))

    fig, ax = g.plot_seats_votes_curve(Dvotes)
    fig.set_size_inches(8, 8)
    n_D_seats = sum(Dvotes>.5)
    plt.scatter(avg, n_D_seats, marker='o', facecolors='none', linewidth=3, s=200, label='number of seats given an average voteshare', color='forestgreen', zorder=100)
    plt.scatter(.5, N/2, label='point at which partisan bias=0 and mean-median=0', color=curvecolor)
    plt.axvline(avg, label='average vote', color=avgcolor)
    if show_hist:
        ax.hist(Dvotes, bins=50, label='election histogram', color='mediumpurple')

    plt.legend()
    plt.ylim([0, N])
    plt.xlim([0, 1])
    if use_percents:
        ax.set_xticklabels([f'{i:.0f}%' for i in np.linspace(0,100,11)])
        ax.set_xlabel('hypothetical Dem vote percentage')
        
        ax.set_yticks(np.linspace(0,1,11)*N)
        ax.set_yticklabels([f'{i:.0f}%' for i in np.linspace(0,100,11)])
        ax.set_ylabel('Dem seat percentage')


layout = wdgt.Layout(width='650px')
style = {'description_width': '400px'}

avg_widget = wdgt.FloatSlider(value=.54, min=0.0, max=1, step=0.01, description='average 2-party voteshare', continuous_update=False, style=style, layout=layout)
avg_widget.style.handle_color = avgcolor
N_widget = wdgt.IntSlider(value=4000, min=40, max=10000, step=10, description='# of districts', continuous_update=False, style=style, layout=layout)
comp_prop_widget = wdgt.FloatSlider(value=.25, min=0.0, max=1, step=0.01, description='proportion of districts that are competitive', continuous_update=False, style=style, layout=layout)
comp_range_widget = wdgt.FloatSlider(value=.05, min=0.0, max=0.4, step=0.01, description='competitiveness is defined as the average ± this:', continuous_update=False, style=style, layout=layout)
hist_widget = wdgt.Checkbox(value=False, description='show vote histogram')
perc_widget = wdgt.Checkbox(value=True, description='use percents rather than proportions')

interactive_plot = wdgt.interactive(make_plot,
                                    avg=avg_widget,
                                    N=N_widget,
                                    comp_prop=comp_prop_widget,
                                    comp_range=comp_range_widget,
                                    show_hist=hist_widget,
                                    use_percents=perc_widget)
output = interactive_plot.children[-1]
output.layout.height = '700px'
interactive_plot

interactive(children=(FloatSlider(value=0.54, continuous_update=False, description='average 2-party voteshare'…

The above plot allows you to play with the parameters of the bill language to see the resulting seats-votes curve, and the percentage of seats that Dems could expect to win given some average voteshare. The default is to use .54, which is the average of the relevant statewide elections since 2012, as we quote from the bill language above.

The critical thing to note is that the bill requires that *at least* 25% of all districts are competitive. If you keep the settings above at default, and increase the number of competitive districts, the Democrats gain seats accordingly, assuming that they continue to win above 50% of the vote.